In [1]:
import os
import shutil

# Define the paths to your folders
base_path = '/glb/hou/pt.sgs/data/ml_ai_us/uspcjc/datasets/topsalt/wsher'
subfolders = ['train', 'val', 'test']
save_dir = '/glb/hou/pt.sgs/data/ml_ai_us/uspcjc/datasets/topsalt/deeplabv3'

# Create output directories if they don't exist
output_image_dir = os.path.join(save_dir, 'images')
output_label_dir = os.path.join(save_dir, 'labels')

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Iterate through each subfolder (train, val, test)
for subfolder in subfolders:
    image_dir = os.path.join(base_path, 'images', subfolder)
    label_dir = os.path.join(base_path, 'labels', subfolder)
    
    # Get all image files in the current subfolder
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    
    # Copy image files to the output images folder
    for image_file in image_files:
        src_image_path = os.path.join(image_dir, image_file)
        dst_image_path = os.path.join(output_image_dir, image_file)
        shutil.copyfile(src_image_path, dst_image_path)
    
    # Get all label files in the current subfolder
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.png')]
    
    # Copy label files to the output labels folder
    for label_file in label_files:
        src_label_path = os.path.join(label_dir, label_file)
        dst_label_path = os.path.join(output_label_dir, label_file)
        shutil.copyfile(src_label_path, dst_label_path)

print(f"Images copied to {output_image_dir}")
print(f"Labels copied to {output_label_dir}")


Images copied to /glb/hou/pt.sgs/data/ml_ai_us/uspcjc/datasets/topsalt/deeplabv3/images
Labels copied to /glb/hou/pt.sgs/data/ml_ai_us/uspcjc/datasets/topsalt/deeplabv3/labels


In [3]:

import torch
import torch.nn as nn
x = torch.randn(10, 19, requires_grad=True, device='cuda')
y = torch.randint(0, 19, (10,), device='cuda')
y[0] = 255

criterion = nn.CrossEntropyLoss(ignore_index=255)
loss = criterion(x, y)
print(loss)

tensor(3.2581, device='cuda:0', grad_fn=<NllLossBackward0>)


In [21]:
import torch
import torch.nn as nn

class DiceLoss(nn.Module):
    def __init__(self, size_average=True):
        super(DiceLoss, self).__init__()
        self.size_average = size_average

    def forward(self, logit, target):
        n, c, h, w = logit.size()
        
        # Apply sigmoid to get probabilities for the binary class
        logit = torch.sigmoid(logit)
        
        # Flatten logits and labels
        logit_flat = logit.view(n, -1)
        target_flat = target.view(n, -1)
        
        # Calculate intersection and union
        intersection = torch.sum(logit_flat * target_flat, dim=1)
        union = torch.sum(logit_flat, dim=1) + torch.sum(target_flat, dim=1) + 1e-8
        
        # Calculate Dice coefficient for the batch
        dice_coefficients = (2.0 * intersection / union)
        
        # Dice Loss is 1 - average Dice coefficient for the batch
        dice_loss = 1.0 - dice_coefficients.mean()
        
        return dice_loss

# Example usage:
n_classes = 1  # Binary classification (foreground vs background)

# Example tensors: a batch of 2 images, each with size 4x4
logit = torch.tensor([[[[0.2, 0.8, 0.1, 0.9], [0.3, 0.7, 0.4, 0.6], [0.5, 0.5, 0.2, 0.8], [0.3, 0.7, 0.4, 0.6]]],
                      [[[0.3, 0.7, 0.3, 0.7], [0.2, 0.8, 0.1, 0.9], [0.4, 0.6, 0.3, 0.7], [0.5, 0.5, 0.2, 0.8]]]])

target = torch.tensor([[[0, 1, 0, 1], [1, 1, 0, 1], [1, 0, 1, 1], [0, 1, 1, 1]],
                       [[1, 0, 1, 1], [0, 1, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0]]])

# Initialize DiceLoss and compute the loss
dice_loss_fn = DiceLoss()
loss = dice_loss_fn(logit, target)
print(f"Dice Loss: {loss.item()}")


Dice Loss: 0.3579835891723633


In [29]:
import torch
import torch.nn as nn

class WeightedDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(WeightedDiceLoss, self).__init__()
        self.weight = weight
        self.size_average = size_average

    def forward(self, logit, target):
        n, c, h, w = logit.size()
        
        # Apply sigmoid to get probabilities for the binary class
        logit = torch.sigmoid(logit)
        
        # Flatten logits and labels
        logit_flat = logit.view(n, -1)
        target_flat = target.view(n, -1)
        
        # Calculate intersection and union
        intersection = torch.sum(logit_flat * target_flat, dim=1)
        union = torch.sum(logit_flat, dim=1) + torch.sum(target_flat, dim=1) + 1e-8
        
        # Calculate Dice coefficient for the batch
        dice_coefficients = (2.0 * intersection / union)
        
        # If weights are provided, apply them to the Dice coefficient
        if self.weight is not None:
            dice_coefficients = dice_coefficients * self.weight
        
        # Dice Loss is 1 - average Dice coefficient for the batch
        dice_loss = 1.0 - dice_coefficients.mean()
        
        return dice_loss

# Example usage:
n_classes = 1  # Binary classification (foreground vs background)

# # Example tensors: a batch of 2 images, each with size 4x4
# logit = torch.tensor([[[[0.2, 0.8, 0.1, 0.9], [0.3, 0.7, 0.4, 0.6], [0.5, 0.5, 0.2, 0.8], [0.3, 0.7, 0.4, 0.6]]],
#                       [[[0.3, 0.7, 0.3, 0.7], [0.2, 0.8, 0.1, 0.9], [0.4, 0.6, 0.3, 0.7], [0.5, 0.5, 0.2, 0.8]]]])

# target = torch.tensor([[[0, 1, 0, 1], [1, 1, 0, 1], [1, 0, 1, 1], [0, 1, 1, 1]],
#                        [[1, 0, 1, 1], [0, 1, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0]]])

# # Initialize WeightedDiceLoss and compute the loss
# weight = 1.0  # Example weight to give more importance to the foreground class
# dice_loss_fn = WeightedDiceLoss(weight=weight)
# loss = dice_loss_fn(logit, target)
# print(f"Weighted Dice Loss: {loss.item()}")


In [47]:
import torch
import torch.nn as nn

class WeightedDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(WeightedDiceLoss, self).__init__()
        self.weight = weight
        self.size_average = size_average

    def forward(self, logit, target):
        n, c, h, w = logit.size()
        
        # Apply sigmoid to get probabilities for the binary class
        logit = torch.sigmoid(logit)
        
        # Flatten logits and labels
        logit_flat = logit.view(n, -1)
        target_flat = target.view(n, -1).float()
        
        # Calculate intersection and union
        intersection = torch.sum(logit_flat * target_flat, dim=1)
        union = torch.sum(logit_flat, dim=1) + torch.sum(target_flat, dim=1) + 1e-8
        
        # Calculate Dice coefficient for the batch
        dice_coefficients = (2.0 * intersection / union)
        
        # If weights are provided, apply them to the Dice coefficient
        if self.weight is not None:
            dice_coefficients = dice_coefficients * self.weight
        
        # Dice Loss is 1 - average Dice coefficient for the batch
        dice_loss = 1.0 - dice_coefficients.mean()
        
        return dice_loss

# Example usage:
n_classes = 1  # Binary classification (foreground vs background)

# Example tensors: a batch of 2 images, each with size 4x4
logit = torch.tensor([[[[0.2, 0.8, 0.1, 0.9], [0.3, 0.7, 0.4, 0.6], [0.5, 0.5, 0.2, 0.8], [0.3, 0.7, 0.4, 0.6]]],
                      [[[0.3, 0.7, 0.3, 0.7], [0.2, 0.8, 0.1, 0.9], [0.4, 0.6, 0.3, 0.7], [0.5, 0.5, 0.2, 0.8]]]])

target = torch.tensor([[[0, 1, 0, 1], [1, 1, 0, 1], [1, 0, 1, 1], [0, 1, 1, 1]],
                       [[1, 0, 1, 1], [0, 1, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0]]])

# Add channel dimension to the target tensor
target = target.unsqueeze(1)

# Initialize WeightedDiceLoss and compute the loss
weight = 1.0  # Example weight to give more importance to the foreground class
dice_loss_fn = WeightedDiceLoss(weight=weight)
loss = dice_loss_fn(logit, target)
print(f"Weighted Dice Loss: {loss.item()}")



Weighted Dice Loss: 0.3579835891723633


In [50]:
import torch
import torch.nn.functional as F
from torch import nn as nn
from torch.nn import MSELoss, SmoothL1Loss, L1Loss


def flatten(tensor):
    """Flattens a given tensor such that the channel axis is first.
    The shapes are transformed as follows:
       (N, C, D, H, W) -> (C, N * D * H * W)
    """
    # number of channels
    C = tensor.size(1)
    # new axis order
    axis_order = (1, 0) + tuple(range(2, tensor.dim()))
    # Transpose: (N, C, D, H, W) -> (C, N, D, H, W)
    transposed = tensor.permute(axis_order)
    # Flatten: (C, N, D, H, W) -> (C, N * D * H * W)
    return transposed.contiguous().view(C, -1)

def compute_per_channel_dice(input, target, epsilon=1e-6, weight=None):
    """
    Computes DiceCoefficient as defined in https://arxiv.org/abs/1606.04797 given  a multi channel input and target.
    Assumes the input is a normalized probability, e.g. a result of Sigmoid or Softmax function.

    Args:
         input (torch.Tensor): NxCxSpatial input tensor
         target (torch.Tensor): NxCxSpatial target tensor
         epsilon (float): prevents division by zero
         weight (torch.Tensor): Cx1 tensor of weight per channel/class
    """

    # input and target shapes must match
    assert input.size() == target.size(), "'input' and 'target' must have the same shape"

    input = flatten(input)
    target = flatten(target)
    target = target.float()

    # compute per channel Dice Coefficient
    intersect = (input * target).sum(-1)
    if weight is not None:
        intersect = weight * intersect

    # here we can use standard dice (input + target).sum(-1) or extension (see V-Net) (input^2 + target^2).sum(-1)
    denominator = (input * input).sum(-1) + (target * target).sum(-1)
    return 2 * (intersect / denominator.clamp(min=epsilon))


class _AbstractDiceLoss(nn.Module):
    """
    Base class for different implementations of Dice loss.
    """

    def __init__(self, weight=None, normalization='sigmoid'):
        super(_AbstractDiceLoss, self).__init__()
        self.register_buffer('weight', weight)
        # The output from the network during training is assumed to be un-normalized probabilities and we would
        # like to normalize the logits. Since Dice (or soft Dice in this case) is usually used for binary data,
        # normalizing the channels with Sigmoid is the default choice even for multi-class segmentation problems.
        # However if one would like to apply Softmax in order to get the proper probability distribution from the
        # output, just specify `normalization=Softmax`
        assert normalization in ['sigmoid', 'softmax', 'none']
        if normalization == 'sigmoid':
            self.normalization = nn.Sigmoid()
        elif normalization == 'softmax':
            self.normalization = nn.Softmax(dim=1)
        else:
            self.normalization = lambda x: x

    def dice(self, input, target, weight):
        # actual Dice score computation; to be implemented by the subclass
        raise NotImplementedError

    def forward(self, input, target):
        # get probabilities from logits
        input = self.normalization(input)

        # compute per channel Dice coefficient
        per_channel_dice = self.dice(input, target, weight=self.weight)

        # average Dice score across all channels/classes
        return 1. - torch.mean(per_channel_dice)


class DiceLoss(_AbstractDiceLoss):
    """Computes Dice Loss according to https://arxiv.org/abs/1606.04797.
    For multi-class segmentation `weight` parameter can be used to assign different weights per class.
    The input to the loss function is assumed to be a logit and will be normalized by the Sigmoid function.
    """

    def __init__(self, weight=None, normalization='sigmoid'):
        super().__init__(weight, normalization)

    def dice(self, input, target, weight):
        return compute_per_channel_dice(input, target, weight=self.weight)

In [51]:
N, C, D, H, W = 2, 3, 4, 5, 6  # Example dimensions: Batch size, Channels, Depth, Height, Width
input_tensor = torch.randn(N, C, D, H, W, requires_grad=True)  # Random logits
target_tensor = torch.randint(0, 2, (N, C, D, H, W)).float()   # Binary target tensor

# Step 2: Initialize DiceLoss class
dice_loss = DiceLoss(normalization='sigmoid')

# Step 3: Compute loss
loss = dice_loss(input_tensor, target_tensor)

# Print the result
print("Dice Loss:", loss.item())

Dice Loss: 0.3459131717681885


In [53]:
import torch
import torch.nn.functional as F
from torch import nn

# Assuming the previous code defining flatten, compute_per_channel_dice, _AbstractDiceLoss, and DiceLoss is in the same script or imported correctly

# Define the logit and target tensors as provided
logit = torch.tensor([[[[0.2, 0.8, 0.1, 0.9], 
                        [0.3, 0.7, 0.4, 0.6], 
                        [0.5, 0.5, 0.2, 0.8], 
                        [0.3, 0.7, 0.4, 0.6]]],

                      [[[0.3, 0.7, 0.3, 0.7], 
                        [0.2, 0.8, 0.1, 0.9], 
                        [0.4, 0.6, 0.3, 0.7], 
                        [0.5, 0.5, 0.2, 0.8]]]])

target = torch.tensor([[[0, 1, 0, 1], 
                        [1, 1, 0, 1], 
                        [1, 0, 1, 1], 
                        [0, 1, 1, 1]],

                       [[1, 0, 1, 1], 
                        [0, 1, 1, 0], 
                        [1, 1, 0, 0], 
                        [1, 1, 1, 0]]])

# Add a channel dimension to target to match the expected shape (N, C, H, W)
target = target.unsqueeze(1).float()

# Initialize the DiceLoss class
dice_loss = DiceLoss(normalization='sigmoid')

# Compute the loss
loss = dice_loss(logit, target)

# Print the result
print("Dice Loss:", loss.item())

# # Optional: Backward pass to check gradient computation
# loss.backward()
# print("Gradients:", logit.grad)


Dice Loss: 0.21425658464431763
